extract models from fast5 files. 

In [1]:
import h5py
import re
import pandas
import numpy as np
from pandas.util.testing import assert_frame_equal
import pickle

In [2]:
args = {
    "fast5": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/LomanLab_MAP-006/MAP006-1_5000/",
    "out_basename": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1"
}
# args = {
#     "fast5": "/home/ibis/gregor.sturm/nanopore/NanoporeData/SequencingRuns/NanoporeData/burnin_downloads/",
#     "out_basename": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/ibislambda"
# }
# args = {
#     "fast5": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/david_eccles_bc_ideas/",
#     "out_basename": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/davidmouse"
# }

In [3]:
fast5path = !find {args["fast5"]} | grep fast5

In [4]:
models = []
models_params = {}
for file in fast5path:
    f5 = h5py.File(file, 'r')
    try:
        log = f5['/Analyses/Basecall_2D_000/Log']
        log = bytes(log[...]).decode("utf-8").split("\n")
        model = [x for x in log if x.find(".model") >= 0]
        model = [re.search(r"\"(.*)\"", x).group(1) for x in model]
        models.extend(model)
    except KeyError: 
        print("Error opening {0}".format(file))
        continue
    
    for t in ["template", "complement"]: 
        try:
            m_name = [x for x in model if x.find(t) >= 0][0]
            params = pandas.DataFrame(np.array(f5['/Analyses/Basecall_2D_000/BaseCalled_{0}/Model'.format(t)]))
            try:
                assert_frame_equal(models_params[m_name], params)
            except KeyError:
                """model not loaded yet"""
                models_params[m_name] = params
        except IndexError:
            """model not available in this file"""
            pass
        
        
    

In [5]:
set(models)

{'/opt/chimaera/model/r7.3_e6_70bps_6mer/complement_median68pA_pop1.model',
 '/opt/chimaera/model/r7.3_e6_70bps_6mer/complement_median68pA_pop2.model',
 '/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model'}

In [6]:
models_params

{'/opt/chimaera/model/r7.3_e6_70bps_6mer/complement_median68pA_pop1.model':            kmer  level_mean  level_stdv   sd_mean   sd_stdv      weight
 0     b'AAAAAA'   63.850362    0.935038  0.738710  0.250211  446.495548
 1     b'AAAAAC'   58.825628    0.902580  0.820361  0.292821  484.581081
 2     b'AAAAAG'   64.144299    0.896533  0.768197  0.265341  350.351867
 3     b'AAAAAT'   62.349891    0.994614  0.923557  0.349777  410.596197
 4     b'AAAACA'   58.059086    1.184754  0.849544  0.308585  319.995660
 5     b'AAAACC'   55.258320    0.887990  0.934303  0.355899  488.912406
 6     b'AAAACG'   57.424781    0.982879  0.795935  0.279841  445.628815
 7     b'AAAACT'   57.192332    1.048211  0.748799  0.255354  232.701578
 8     b'AAAAGA'   64.153616    1.330809  0.829369  0.297657  200.282748
 9     b'AAAAGC'   59.252629    0.903662  0.903942  0.338693  385.262916
 10    b'AAAAGG'   63.041242    0.846662  0.784827  0.274003  189.864401
 11    b'AAAAGT'   61.717141    0.831023  1.05173

In [7]:
for params in models_params.values():
    params["kmer"] = params["kmer"].map(lambda x: x.decode("utf-8"))

In [8]:
pickle.dump(models_params, open(args["out_basename"] + ".model.pickle", 'wb'), protocol=2)

In [9]:
for name, params in sorted(models_params.items()): 
    print(name, params["level_mean"].median(), params["level_mean"].mean())

/opt/chimaera/model/r7.3_e6_70bps_6mer/complement_median68pA_pop1.model 60.256377 59.26414875146482
/opt/chimaera/model/r7.3_e6_70bps_6mer/complement_median68pA_pop2.model 61.765499000000005 60.37976793847641
/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model 59.101268000000005 58.14811564526377
